# Question Answering

Recall the overall workflow for retrieval augmented generation

![](rag.jpeg)

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`. Now we'll abord `Question Answering`.

Let's load our vectorDB. 

In [1]:
import os
import openai

In [2]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [3]:
print(vectordb._collection.count())

209


In [4]:
question = "what are major topics for this class?"
docs = vectordb.similarity_search(question, k=3)
len(docs)

3

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)

## RetrievalQA Chain

In [6]:
from langchain.chains import RetrievalQA

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [8]:
result = qa_chain({"query": question})

In [9]:
result["result"]

'The major topics for this class include machine learning, statistics, and algebra. Additionally, there will be discussions on extensions of the material covered in the main lectures.'